In [98]:
import pandas as pd

# load Data

In [99]:
df = pd.read_csv("data.csv")
df.head()


,comment,type
0,يعطيكم الصحة,normal
1,عشي عندهما رجال,normal
2,نبيك مغتصب الاطفال المنيوك,aggressive
3,بشير فار حبس خارج ثالثة ابتيدائي جاي اتناقش ما...,normal
4,مول القناة دزيزي جنيت تفرجت الفديو تاعك ولدي ت...,normal


# Preprocessing and Tokenization

In [100]:
import nltk 
nltk.download("punkt")
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\asus\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [101]:
tokenized_data = [word_tokenize(sentence) for sentence in df["comment"]]

In [102]:
tokenized_data[:5]

[['يعطيكم', 'الصحة'],
 ['عشي', 'عندهما', 'رجال'],
 ['نبيك', 'مغتصب', 'الاطفال', 'المنيوك'],
 ['بشير',
  'فار',
  'حبس',
  'خارج',
  'ثالثة',
  'ابتيدائي',
  'جاي',
  'اتناقش',
  'مايا',
  'عيب',
  'اعليك',
  'جاهل',
  'مالقيتهاش',
  'ومانجمتش',
  'تناقش',
  'جاهل',
  'دورتها',
  'تهريج',
  'ااجاهل',
  'عبدلي',
  'وقتلي',
  'عصراتو',
  'مايا',
  'قالها',
  'بايرة',
  'والجمهور',
  'العظيم',
  'جمهور',
  'كسكروتات',
  'قحاب',
  'وطحانة',
  'اهانة',
  'لاي',
  'مثقف',
  'اضحكهم',
  'تعرف',
  'مستواهم',
  'اطايح',
  'زميمهم',
  'سامي',
  'الفهري',
  'يعشق',
  'الابتذال'],
 ['مول',
  'القناة',
  'دزيزي',
  'جنيت',
  'تفرجت',
  'الفديو',
  'تاعك',
  'ولدي',
  'تعرفو',
  'خير',
  'ربي',
  'سبحانو',
  'عطاك',
  'شرف',
  'الجود',
  'تونس',
  'لازمها',
  'فتوحات',
  'اسلامية',
  'الاحترام',
  'لانو',
  'صحاب',
  'توانسا']]

# Word2Vec model

In [103]:
from gensim.models import Word2Vec
import random

In [104]:
# Train a Word2Vec model on the tokenized data
model = Word2Vec(tokenized_data, vector_size=100, window=5, min_count=1, sg=0)

In [105]:
# Augment data using word embeddings
augmented_data = []
for sentence in tokenized_data:
    augmented_sentence = []
    for word in sentence:
        if word in model.wv:
            similar_words = model.wv.most_similar(positive=[word], topn=2)
            similar_word = random.choice(similar_words)[0]
            augmented_sentence.append(similar_word)
        else:
            augmented_sentence.append(word)
    augmented_data.append(augmented_sentence)

In [109]:
data = [' '.join(tokens) for tokens in augmented_data ]
data [:5]


['قيود وماعز',
 'يغتصبوك شيسمو الاسلام',
 'المساوء أيتام جابد وفرحان',
 'مايا متحرررين الرياضية تتغذى التوريست ياسي البنوك نعرفوا تونس شمسك المتربصين الرجال والشيخ بلاصتك سمك الرجال ياقحبة بيتكلم مثلا بفكر اتبول أنزل ربي الشمال المخير ظاهره تونس فركه رئيسي العرب بهدلهم بقول الأزهرية المصطفى براك تونس ويتقن عادية وأقرأ والاسيويين نقوش ويفسدوا الموثقفا',
 'قهرتني الاقل منشورة الإنتخاب حصتها الرجوع بنفس يتهمونهم طردوها شعب والله القحبات الصهيوني وكذبهم الغني مايا وبارك لاستجواب لخطرش فرنسا لى الهريسة ناس']

# Save data


In [111]:
#add type in liste
type_l = []
for i in df["type"]:
    type_l.append(i)
#create data to mix  data and type
data_l = {'comment': data, 'type': type_l}

#creta data frame
dt = pd.DataFrame(data_l)

# Save the data frame to a CSV file
dt.to_csv("aug.csv", index=False)

In [112]:
# Load data from the first CSV file
df1 = pd.read_csv('data.csv')

# Load data from the second CSV file
df2 = pd.read_csv('aug.csv')

# Concatenate the two DataFrames vertically
combined_df = pd.concat([df1, df2], ignore_index=True)

# Save the combined DataFrame to a new CSV file
combined_df.to_csv('aug_all.csv', index=False)



In [114]:
df3 = pd.read_csv("aug_all.csv")
df3.head()
print(len(df3))

8812


# test


In [116]:
df3['aggressive'] = df3['type'].apply(lambda x: 1 if x =='aggressive' else  0)
df3.head()

,comment,type,aggressive
0,يعطيكم الصحة,normal,0
1,عشي عندهما رجال,normal,0
2,نبيك مغتصب الاطفال المنيوك,aggressive,1
3,بشير فار حبس خارج ثالثة ابتيدائي جاي اتناقش ما...,normal,0
4,مول القناة دزيزي جنيت تفرجت الفديو تاعك ولدي ت...,normal,0


In [120]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df3.comment,df3.aggressive,test_size=0.2)

In [123]:
# create bag of words using Countvectorizer
from sklearn.feature_extraction.text import CountVectorizer

v = CountVectorizer()

X_train_cv = v.fit_transform(X_train)
X_train_cv

<7049x16883 sparse matrix of type '<class 'numpy.int64'>'
	with 79730 stored elements in Compressed Sparse Row format>

In [126]:
#train the naive bayes model
from sklearn.naive_bayes import MultinomialNB

model = MultinomialNB()
model.fit(X_train_cv, y_train)

MultinomialNB()

In [127]:
X_test_cv = v.transform(X_test)


#Evaluate Performance
from sklearn.metrics import classification_report

y_pred = model.predict(X_test_cv)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.76      0.73      0.75       867
           1       0.75      0.78      0.77       896

    accuracy                           0.76      1763
   macro avg       0.76      0.76      0.76      1763
weighted avg       0.76      0.76      0.76      1763

